### Libraries

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import yaml
# Local Module Imports
import sys
import importlib

sys.path.insert(0, "../src")
import visualization

importlib.reload(visualization)
from visualization import create_boxplot

import utils

importlib.reload(utils)
from utils import get_files_dirs_ext


In [31]:
# Set base directory
BASE_DIR = os.path.dirname(os.getcwd())
# model output directory
OUTPUT_DIR = os.path.join(BASE_DIR, "output")

# Load 'config' file
config_file = os.path.join(BASE_DIR, "config.yml")
with open(config_file, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

In [9]:
df_int_iou = pd.read_csv('interpol_iou.csv')
df_int_hd = pd.read_csv('interpol_hd.csv')
df_ext_70_hd = pd.read_csv('extrapol_70_hd.csv')
df_ext_90_hd = pd.read_csv('extrapol_90_hd.csv')
df_ext_70_iou = pd.read_csv('extrapol_70_iou.csv')
df_ext_90_iou = pd.read_csv('extrapol_90_iou.csv')

In [16]:
df_int_iou["hd_list"]

0    [0.8866869554117072, 0.9079692754977028, 0.896...
1    [0.9252892156155551, 0.9413981610619676, 0.933...
2    [0.8685295915661158, 0.8924251652115361, 0.891...
3    [0.9054595207782747, 0.9299367498003063, 0.921...
4    [0.8028163202106775, 0.8259772608769321, 0.819...
5    [0.8284797621854328, 0.8039887716053437, 0.803...
Name: hd_list, dtype: object

In [32]:
csv_files = get_files_dirs_ext(dirs=OUTPUT_DIR, ext="csv", return_paths=True)
csv_files

In [23]:
csv_files

[]